In [1]:
from brainlit.utils.session import NeuroglancerSession
from brainlit.utils.swc import graph_to_paths
import napari

/Users/johnduva/opt/anaconda3/envs/neuroData3_7/lib/python3.7/site-packages/python_jsonschema_objects/__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


In [7]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"
mip = 2
radius = 15 #should this be 75 as in the tutorial? or 15 because Bijan
v_id = 0

# Create a list of imgs, bboxes, voxs for each cell in 'dir' using 'pull_voxel'
imgs = []
bboxs = []
voxs = []
IDs = []
cell_centers = []
ngl_sess = NeuroglancerSession(mip=mip, url=dir, url_segments=dir_segments)
for seg_id in range(1000):
    try:
        # Get 0th voxel coordinates (cell center)
        # [x,y,z] = ngl_sess._get_voxel(seg_id, 0)
        cell_centers.append(ngl_sess._get_voxel(seg_id, 0))
        
        img, bbox, vox = ngl_sess.pull_voxel(seg_id, v_id, radius) 
        # print(f"\n\nDownloaded volume is of shape {img.shape}, with individual total intensities of {sum(sum(sum(img)))}.")
        imgs.append(img)
        bboxs.append(bbox)
        voxs.append(vox)
        IDs.append(seg_id)
    except:
        pass

Downloading: 100%|██████████| 1/1 [00:00<00:00, 19.87it/s]


Downloading:   0%|          | 0/4 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 25.92it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  5.50it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 28.80it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 28.48it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 30.02it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 31.88it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 24.34it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 29.06it/s]


In [18]:
count = 0;
paths = []
for seg_id in IDs:    
    G_sub = ngl_sess.get_segments(seg_id, bboxs[count])
    path = graph_to_paths(G_sub)
    paths.append(path) 
    # print(f"Selected volume contains {G_sub.number_of_nodes()} nodes and {len(path)} paths")
    count+=1

Downloading: 100%|██████████| 1/1 [00:00<00:00, 16.13it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  8.52it/s]


In [23]:
# for i,path in enumerate(paths):
i = 3
path = paths[i]
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image( imgs[i] )
    viewer.add_shapes(data=path, shape_type='path', edge_width=0.1, edge_color='blue', opacity=0.1)
    viewer.add_points([15,15,15], size=1, opacity=0.5)

In [16]:
import csv
import pandas as pd
filename = 'cellCenters4.csv'
with open(filename, mode='w') as employee_file:
    for i in range(0, len(cell_centers)):
        cntr = cell_centers[i]
        employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        employee_writer.writerow([i, cntr[0], cntr[1], cntr[2]])

# Rename the columns appropriately
df = pd.read_csv(filename, header=None)
df.rename(columns={0: 'Id',  1: 'Center of Mass (Geometry): X (px)', 2: 'Center of Mass (Geometry): Y (px)', 3: 'Center of Mass (Geometry): Z (px)'}, inplace=True)
df.to_csv(filename, index=False) # save to new csv file